In [7]:
import pandas as pd
import torch
import os
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

In [2]:
result_dir = "H:/JHMDB/models"

In [9]:
results = {}
for m in os.listdir(f"{result_dir}"):

    print(m)

    max_acc = 0

    for e in os.listdir(f"{result_dir}/{m}"):

        res = torch.load(f"{result_dir}/{m}/{e}", map_location=torch.device('cpu'))
        acc = accuracy_score(res['val_actual'], res['val_predicted'])

        if acc > max_acc:
            max_acc = acc

    results[m].append(max_acc)

sorted = {k: v for k, v in sorted(results.items(), key=lambda item: item[1])}
for k in sorted.keys():
    print(f"{k} - {sorted[k]}")

m_test


KeyboardInterrupt: 